In [20]:
import numpy as np
import torch
import pandas as pd
import pickle as pkl
import operator

In [2]:
data = pd.read_csv("data/pair_sentences.csv", sep="|", index_col=0)

/home/lucas/anaconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
data

,previous_sentence,sentence,label
0,The Benefits of a Sydney Corporate Fitness Pro...,"Maybe if we eliminated the minimum wage, peopl...",1
1,Were chickens firing rockets?,Well we do know that geese have taken down a p...,1
2,"Adobe, Apple working together on Flash for iPhone",Considering the MacOS X plug-in is a complete ...,0
3,where Workers take the most sick days.,Other indicator of America's superior health c...,1
4,I've been searching for the answer for this fo...,It's obviously tracks from a giant water tract...,0
5,It's obviously tracks from a giant water tract...,obviously its the folds in the google map,0
6,obviously its the folds in the google map,Answer: It's a quilt.,0
7,Answer: It's a quilt.,Lupus.,0
8,Lupus.,It's a tell-tale sign where the 3rd Stargate i...,0
9,It's a tell-tale sign where the 3rd Stargate i...,I think they got lazy and scanned a map.,0


## For now we take the train set and divide it into train - val - test
## Letter the test set will be the same as in the paper

In [4]:
train = data.ix[:3964297]
val = data.ix[3964297:5285729]
test = data.ix[5285729:]

/home/lucas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [30]:
class Dictionary(object):
    def __init__(self):
        self.unknown = "<UNK>"
        self.wordCount = {}
        self.word2idx = {self.unknown:0}
        self.idx2word = [self.unknown]
        
    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        
        self.wordCount[word] = self.wordCount.get(word, 0) + 1
        return self.word2idx[word]
    
    def reoder(self):
        # the word index is dependent on its number of occurence, for triming voc.
        sorted_x = sorted(self.wordCount.items(), key=operator.itemgetter(1), reverse=True)
        new_word2idx = {self.unknown:0}
        new_idx2word = [self.unknown]
        
        for key_value in sorted_x:
            word = key_value[0]
            new_idx2word.append(word)
            new_word2idx[word] = len(self.idx2word) - 1
            
        self.word2idx = new_word2idx
        self.idx2word = new_idx2word
            
    def get_index(self, word):
        return self.word2idx.get(word, self.unknown)
        
    def __len__(self):
        return len(self.idx2word)

class Corpus(object):
    def __init__(self, train, val, test):
        self.dictionary = Dictionary()
        self.train = train
        self.valid = val
        self.test = test

    def tokenize(self):
        """Get all words in train set, set unseen words in val / test to UNK."""
        for row in range(self.train.shape[0]):
            elem = self.train.ix[row]
            words = str(elem["previous_sentence"]).split(" ") + ['<eos>'] + str(elem["sentence"]).split(" ") + ['<eos>']
            
            for word in words:
                self.dictionary.add_word(word)
                    
    def write(self, path, dataset):        
        with open(path, 'w+') as f:
            for row in range(dataset.shape[0]):
                elem = self.train.ix[row]
                line = ""
                words = str(elem["previous_sentence"]).split() + ['<eos>']
                for word in words:
                    line += str(self.dictionary.get_index(word)) + " "
                    
                line += ", "
                
                words = str(elem["sentence"]).split() + ['<eos>']
                for word in words:
                    line += str(self.dictionary.get_index(word)) + " "
                
                line += ", "
                line += str(elem["label"])
                f.write(line + "\n")

In [31]:
corpus = Corpus(train, val, test)

In [32]:
print("Tokenize ... ")
# self.tokenize()
corpus.dictionary.wordCount = pkl.load(open('data/rnn/wordcount.pkl', 'r'))
corpus.dictionary.idx2word = pkl.load(open('data/rnn/idx2word.pkl', 'r'))
# corpus.dictionary.word2idx = pkl.load(open('data/rnn/word2idx.pkl', 'r'))

Tokenize ... 


In [33]:
word2idx = {}
for idx in range(len(corpus.dictionary.idx2word)):
    word2idx[corpus.dictionary.idx2word[idx]] = idx

In [34]:
corpus.dictionary.word2idx = word2idx

In [37]:
# corpus.dictionary.reoder()
print("Write train ...")
corpus.write("data/rnn/train-tokenized.txt", corpus.train)

Write train ...


In [36]:
print("Write val + test ...")
corpus.write("data/rnn/val-tokenized.txt", corpus.valid)
print("Test ...")
corpus.write("data/rnn/test-tokenized.txt", corpus.test)

Write val + test ...
Test ...


In [21]:
pkl.dump(word2idx, open('data/rnn/word2idx.pkl', 'w+'))

In [5]:
corpus.dictionary.wordCount.get('bed', 0)

NameError: name 'corpus' is not defined

In [41]:
corpus.dictionary.word2idx['tile']

15987

In [38]:
criterion = nn.BCELoss()

NameError: name 'nn' is not defined